In [1]:
import evaluate
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

In [2]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

 ## 1. Выбор и загрузка набора данных

 Для решения задачи классификации текстов выбран датасет IMDB Reviews.
 Этот набор данных содержит отзывы о фильмах с бинарной разметкой sentiment (positive/negative).

 В данной работе используется уменьшенная версия датасета (4000 примеров для обучения и 1000 для тестирования) для оптимизации времени выполнения на доступных вычислительных ресурсах. При наличии более мощного GPU или большего времени на обучение было бы целесообразно использовать полный набор данных.

In [3]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].shuffle(seed=42).select(range(4000))
test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))
dataset_sampled = {"train": train_dataset, "test": test_dataset}

 ## 2. Выбор и подготовка модели

 В качестве базовой модели выбран DistilBERT - облегченная версия BERT,
 которая сохраняет 97% производительности оригинальной модели, но работает на 60% быстрее.

In [4]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def tokenize_function(examples):
    return tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=256
    )

In [5]:
tokenized_datasets = {}
for split in dataset_sampled:
    tokenized_datasets[split] = dataset_sampled[split].map(
        tokenize_function,
        batched=True,
        remove_columns=[
            col for col in dataset_sampled[split].column_names if col not in ["label"]
        ],
    )

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

 ## 3. Определение метрик качества

 Для оценки качества модели используются accuracy и F1-score.

In [7]:
accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)[
            "accuracy"
        ],
        "f1": f1_score.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"],
    }

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 ## 4. Оценка исходной модели
 Измерение качества предобученной модели DistilBERT на тестовой выборке IMDB Reviews до выполнения fine-tuning.

In [9]:
trainer = Trainer(
    model=model,
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

initial_metrics = trainer.evaluate()
print("Initial metrics:", initial_metrics)

/tmp/ipykernel_64956/2559754658.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Initial metrics: {'eval_loss': 0.6967155933380127, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.408, 'eval_f1': 0.404652143178459, 'eval_runtime': 10.3939, 'eval_samples_per_second': 96.211, 'eval_steps_per_second': 12.026}


 ## 5. Дообучение модели

 Процесс дообучения (fine-tuning) производится на выборке из 4000 размеченных примеров
 из датасета IMDB. Используются следующие параметры:
 - Скорость обучения (learning rate): 2e-5
 - Размер батча: 32
 - Количество эпох: 2
 - L2-регуляризация (weight decay): 0.01

 На каждой эпохе производится оценка качества на валидационной выборке.

In [10]:
training_args = TrainingArguments(
    output_dir="distilbert-imdb-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/reveur/anaconda3/envs/itmo_dl_course/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_64956/2691310023.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.369700,0.329480,0.858000,0.857508
2,0.212800,0.292642,0.871000,0.871008


TrainOutput(global_step=250, training_loss=0.3525660514831543, metrics={'train_runtime': 636.8411, 'train_samples_per_second': 12.562, 'train_steps_per_second': 0.393, 'total_flos': 529869594624000.0, 'train_loss': 0.3525660514831543, 'epoch': 2.0})

In [11]:
final_metrics = trainer.evaluate()
print("\nComparison of metrics:")
print("Before fine-tuning:", initial_metrics)
print("After fine-tuning:", final_metrics)


Comparison of metrics:
Before fine-tuning: {'eval_loss': 0.6967155933380127, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.408, 'eval_f1': 0.404652143178459, 'eval_runtime': 10.3939, 'eval_samples_per_second': 96.211, 'eval_steps_per_second': 12.026}
After fine-tuning: {'eval_loss': 0.29264160990715027, 'eval_accuracy': 0.871, 'eval_f1': 0.8710081270731437, 'eval_runtime': 24.1998, 'eval_samples_per_second': 41.323, 'eval_steps_per_second': 1.322, 'epoch': 2.0}


 ## 6. Анализ результатов

 ### Метрики до fine-tuning:
 - Accuracy: 0.408 (40.8%)
 - F1-score: 0.405 (40.5%)

 ### Метрики после fine-tuning:
 - Accuracy: 0.871 (87.1%)
 - F1-score: 0.871 (87.1%)

 ### Выводы:
 1. Исходная модель показала низкое качество классификации (около 40%), что ожидаемо, так как она не была обучена на данной задаче.
 2. После дообучения качество значительно улучшилось - обе метрики достигли 87%.
 3. Равенство метрик accuracy и F1-score говорит о сбалансированности классов в датасете.
 4. Достигнутый результат можно считать успешным для задачи бинарной классификации текстов.

 Таким образом, fine-tuning предобученной модели DistilBERT позволил успешно адаптировать её для решения
 задачи классификации отзывов IMDB.